# ChromaDB Semantic Search 

ChromaDB is a vector database that makes it easy to build semantic search systems

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

In [3]:
# Initialize an in-memory Chroma client
print("Initializing Chroma client...")
client = chromadb.Client()

# Create embedding function using SentenceTransformer
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

Initializing Chroma client...


## Creating Collections

In ChromaDB, documents are organized into collections

In [4]:
collection = client.create_collection(
    name="documents",
    embedding_function=embedding_function
)

In [5]:
# Helper function to display search results in a readable format
def display_results(results):
    """ Display ChromaDB search results """
    print("Results : ")
    for i,(doc,doc_id, metadata, distance) in enumerate(zip(
        results["documents"][0], 
        results["ids"][0],
        results["metadatas"][0] if results["metadatas"] else [None * len(results["ids"][0])],
        results["distances"][0])):
        
        print(f"{i + 1}. Document : {doc}")
        print(f"    ID: {doc_id}")
        if metadata:
            print(f"    Metadata : {metadata}")
        print(f"    Distance : {distance:.4f}")
        print()

## Basic Vector Operations

Adding documents and performing simple semantic search

In [6]:
# Example documents covering various topics
documents = [
    "The quick brown fox jumps over the lazy dog",
    "A man is walking his dog in the park",
    "The weather is sunny and warm today",
    "Artificial intelligence is transforming the technology landscape",
    "Vector databases are essential for semantic search applications",
    "Deep learning models require substantial computational resources",
    "The city skyline looks beautiful at sunset",
    "Machine learning algorithms find patterns in data"
]

ids = ["doc1", "doc2", "doc3", "doc4", "doc5", "doc6", "doc7", "doc8"]

# Add documents to collection
print("Adding documents to collection...")
collection.add(documents=documents, ids=ids)

# # Get collection count
count = collection.count()
print(f"Collection now contains {count} documents")

# Perform a semantic search
query_text = "AI and technology trends"
print(f"Performing similarity search for : '{query_text}'")
print()

results = collection.query(
    query_texts=[query_text], 
    n_results=3  # Return top 3 most similar documents
)

# Display results
display_results(results)

Adding documents to collection...
Collection now contains 8 documents
Performing similarity search for : 'AI and technology trends'

Results : 
1. Document : Artificial intelligence is transforming the technology landscape
    ID: doc4
    Distance : 0.3236

2. Document : Machine learning algorithms find patterns in data
    ID: doc8
    Distance : 0.6821

3. Document : Deep learning models require substantial computational resources
    ID: doc6
    Distance : 0.7001



## Working with Metadata and Filtering

ChromaDB allows attaching metadata to documents and filtering searches based on this metadata

In [7]:
# Create a new collection for filtered documents
filtered_doc_collection = client.create_collection(
    name="filtered_documents",
    embedding_function=embedding_function
)

# Metadata for each document
metadatas = [
    {"category": "animal", "length": "short", "year": 2021},
    {"category": "lifestyle", "length": "short", "year": 2022},
    {"category": "weather", "length": "short", "year": 2023},
    {"category": "technology", "length": "medium", "year": 2023},
    {"category": "technology", "length": "medium", "year": 2024},
    {"category": "technology", "length": "long", "year": 2024},
    {"category": "travel", "length": "short", "year": 2023},
    {"category": "technology", "length": "medium", "year": 2024}
]

# Add documents with metadata
print("Adding documents with metadata...")
filtered_doc_collection.add(
    documents=documents,
    ids=ids,
    metadatas=metadatas
)

# Get collection count
count = filtered_doc_collection.count()
print(f"Filtered Docs Collection now contains {count} documents.")

Adding documents with metadata...
Filtered Docs Collection now contains 8 documents.


## Simple Metadata Filtering

In [8]:
# Find technology documents about AI
print("- Filtering by category 'technology' :")
print()
results = filtered_doc_collection.query(
    query_texts=["AI advancements"], 
    n_results=3, 
    where={"category": "technology"}) # Only search technology documents

display_results(results)

- Filtering by category 'technology' :

Results : 
1. Document : Artificial intelligence is transforming the technology landscape
    ID: doc4
    Metadata : {'category': 'technology', 'length': 'medium', 'year': 2023}
    Distance : 0.4330

2. Document : Machine learning algorithms find patterns in data
    ID: doc8
    Metadata : {'length': 'medium', 'category': 'technology', 'year': 2024}
    Distance : 0.6770

3. Document : Deep learning models require substantial computational resources
    ID: doc6
    Metadata : {'year': 2024, 'length': 'long', 'category': 'technology'}
    Distance : 0.6802



## Complex Metadata Filtering

Using logical operators for more advanced filtering

In [9]:
# Complex filtering - technology documents from 2024
print("- Complex filtering (technology documents from 2024) :")
print()
results = filtered_doc_collection.query(
    query_texts=["AI advancements"],
    n_results=3,
    where={"$and": [
        {"category": {"$eq": "technology"}},  # Category must be technology
        {"year": {"$eq": 2024}}               # Year must be 2024
    ]}
)

display_results(results)

- Complex filtering (technology documents from 2024) :

Results : 
1. Document : Machine learning algorithms find patterns in data
    ID: doc8
    Metadata : {'category': 'technology', 'year': 2024, 'length': 'medium'}
    Distance : 0.6770

2. Document : Deep learning models require substantial computational resources
    ID: doc6
    Metadata : {'length': 'long', 'year': 2024, 'category': 'technology'}
    Distance : 0.6802

3. Document : Vector databases are essential for semantic search applications
    ID: doc5
    Metadata : {'length': 'medium', 'year': 2024, 'category': 'technology'}
    Distance : 0.7696



## Content-Based Filtering

In [10]:
# Using where_document to filter by document content
print("- Filtering documents containing 'Artificial intelligence' :")
print()
results = filtered_doc_collection.query(
    query_texts=["AI advancements"],
    n_results=3,
    where_document={"$contains": "Artificial intelligence"} # Content filter
)

display_results(results)

- Filtering documents containing 'Artificial intelligence' :

Results : 
1. Document : Artificial intelligence is transforming the technology landscape
    ID: doc4
    Metadata : {'year': 2023, 'category': 'technology', 'length': 'medium'}
    Distance : 0.4330



## Document Management

ChromaDB provides methods for updating and deleting documents

In [11]:
# Get document by ID
print(f"- Getting document by ID : doc1")
result = collection.get(ids=["doc1"])
print(f"Original document : {result["documents"][0]}")
print()

# Update document
print("- Updated document")
collection.update(ids=["doc1"],
                  documents=["The quick silver fox leaps over the sleepy hound"]
)

# Verify update
result = collection.get(ids=["doc1"])
print(f"Updated document : {result["documents"][0]}")
print()

# Delete document
print("- Deleting document doc2")
collection.delete(ids=["doc2"])
print()

# Verify deletion
count = collection.count()
print(f"Collection now has {count} documents")

- Getting document by ID : doc1
Original document : The quick brown fox jumps over the lazy dog

- Updated document
Updated document : The quick silver fox leaps over the sleepy hound

- Deleting document doc2

Collection now has 7 documents
